In [2]:
from google.colab import drive
drive.mount('/content/drive/MyDrive')

ValueError: Mountpoint must be in a directory that exists

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
faces_zip = "/content/drive/MyDrive/Face.zip"
nonfaces_zip = "/content/drive/MyDrive/Nonhuman_objects.zip"

!mkdir -p /content/dataset/face
!mkdir -p /content/dataset/nonface

!unzip -qo $faces_zip -d /content/dataset/face
!unzip -qo $nonfaces_zip -d /content/dataset/nonface


unzip:  cannot find or open /content/drive/MyDrive/Face.zip, /content/drive/MyDrive/Face.zip.zip or /content/drive/MyDrive/Face.zip.ZIP.
unzip:  cannot find or open /content/drive/MyDrive/Nonhuman_objects.zip, /content/drive/MyDrive/Nonhuman_objects.zip.zip or /content/drive/MyDrive/Nonhuman_objects.zip.ZIP.


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


In [5]:
import os, shutil
import numpy as np

In [6]:
def split_dataset(src_dir, dst_root, class_name, split=(0.7,0.2,0.1)):
    files = os.listdir(src_dir)
    np.random.shuffle(files)

    n_total = len(files)
    n_train = int(split[0] * n_total)
    n_val = int(split[1] * n_total)

    train_files = files[:n_train]
    val_files = files[n_train:n_train+n_val]
    test_files = files[n_train+n_val:]

    for subdir in ["train", "val", "test"]:
        os.makedirs(os.path.join(dst_root, subdir, class_name), exist_ok=True)

    for f in train_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(dst_root, "train", class_name, f))
    for f in val_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(dst_root, "val", class_name, f))
    for f in test_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(dst_root, "test", class_name, f))


In [7]:
split_dataset("/content/dataset/face", "/content/dataset_ready", "face")
split_dataset("/content/dataset/nonface", "/content/dataset_ready", "nonface")


In [8]:
import os
from PIL import Image

data_dir = "/content/dataset_ready"
max_size = (1280, 1280)

def resize_big_images(data_dir, max_size=(1280,1280)):
    for root, dirs, files in os.walk(data_dir):
        for f in files:
            path = os.path.join(root, f)
            try:
                img = Image.open(path)
                if img.size[0] * img.size[1] > 8947848:
                    print(f"Resizing {path} from {img.size}...")
                    img.thumbnail(max_size, Image.Resampling.LANCZOS)
                    img.save(path)
            except Exception as e:
                print(f"Ошибка с {path}: {e}")

resize_big_images(data_dir, max_size=(1280,1280))
